In [1]:
import mlflow
import torch
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as  pd
import os
torch.cuda.empty_cache()

In [2]:
from torch.utils.data import Dataset
import pytorch_lightning as pl
from torch.utils.data import DataLoader

# code from ClimatExML to make batches etc. 
class ClimatExMLLoader(Dataset):
    def __init__(self, lr_glob, hr_glob) -> None:
        self.lr_glob = lr_glob
        self.hr_glob = hr_glob

    def __len__(self):
        return len(self.lr_glob[0])

    def __getitem__(self, idx):
        lr = torch.stack([torch.load(var[idx]) for var in self.lr_glob])
        hr = torch.stack([torch.load(var[idx]) for var in self.hr_glob])     
        return [lr, hr]


class ClimatExMLData(pl.LightningDataModule):
    def __init__(self, data_glob: dict = None, num_workers: int = 24, batch_size: int = None):
        super().__init__()
        self.data_glob = data_glob
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage: str):
        self.test_data = ClimatExMLLoader(self.data_glob['lr_test'], self.data_glob['hr_test'])
        self.train_data = ClimatExMLLoader(self.data_glob['lr_train'], self.data_glob['hr_train'])

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, pin_memory=True),

    def test_dataloader(self):
        # For some reason this can't be a dictionary?
        return DataLoader(self.test_data, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False, pin_memory=True),

/Users/susanbeale/anaconda3/envs/testing/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/susanbeale/anaconda3/envs/testing/lib/python3.10/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: /Users/susanbeale/anaconda3/envs/testing/lib/python3.10/site-packages/torchvision/image.so
  Reason: image not found
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# loading data 

lr_data_6 = {
    "lr_test": ['/Volumes/LaCie/2year_scale_factor_4/test/uas/lr/*.pt','/Volumes/LaCie/2year_scale_factor_4/test/vas/lr/*.pt'],
    "lr_train": ['/Volumes/LaCie/2year_scale_factor_4/train/uas/lr/*.pt','/Volumes/LaCie/2year_scale_factor_4/train/vas/lr/*.pt']
}

hr_data_6 = {
    "hr_test": ['/Volumes/LaCie/2year_scale_factor_4/test/hs/hr/*.pt'],
    "hr_train": ['/Volumes/LaCie/2year_scale_factor_4/train/hs/hr/*.pt']
}
data_6 = {
    "lr_train": [glob.glob(path) for path in lr_data_6["lr_train"]],
    "hr_train": [glob.glob(path) for path in hr_data_6["hr_train"]],
    "lr_test": [glob.glob(path) for path in lr_data_6["lr_test"]],
    "hr_test": [glob.glob(path) for path in hr_data_6["hr_test"]]
}


In [4]:
lr_data_16 = {
    "lr_test": ['/Volumes/LaCie/2year_coarsened_cropped/test/uas/lr/*.pt','/Volumes/LaCie/2year_coarsened_cropped/test/vas/lr/*.pt'],
    "lr_train": ['/Volumes/LaCie/2year_coarsened_cropped/train/uas/lr/*.pt','/Volumes/LaCie/2year_coarsened_cropped/train/vas/lr/*.pt']
}

hr_data_16 = {
    "hr_test": ['/Volumes/LaCie/2year_coarsened_cropped/test/hs/hr/*.pt'],
    "hr_train": ['/Volumes/LaCie/2year_coarsened_cropped/train/hs/hr/*.pt']
}
data_16 = {
    "lr_train": [glob.glob(path) for path in lr_data_16["lr_train"]],
    "hr_train": [glob.glob(path) for path in hr_data_16["hr_train"]],
    "lr_test": [glob.glob(path) for path in lr_data_16["lr_test"]],
    "hr_test": [glob.glob(path) for path in hr_data_16["hr_test"]]
}

In [5]:
# creates a list of tensors test_data[time stamp][input =0, output=1]
test_data_6 = ClimatExMLLoader(data_6['lr_test'], data_6['hr_test'])
input_size_6 = test_data_6[0][0].size()
output_size_6 = test_data_6[0][1].size()
print('input size: ', input_size_6, 'output size: ', output_size_6)

input size:  torch.Size([2, 32, 32]) output size:  torch.Size([1, 128, 128])


In [6]:
test_data_16 = ClimatExMLLoader(data_16['lr_test'], data_16['hr_test'])
input_size_16 = test_data_16[0][0].size()
output_size_16 = test_data_16[0][1].size()
print('input size: ', input_size_16, 'output size: ', output_size_16)

input size:  torch.Size([2, 16, 16]) output size:  torch.Size([1, 128, 128])


In [7]:
# making arrays of data for input 

wind_6 = []
hs_6 = []

for i in range(len(test_data_6)):
    wind_6.append(test_data_6[i][0].detach().numpy())
    hs_6.append(test_data_6[i][1].detach().numpy())
    
w_6 = np.array(wind_6)
# print(w[1])
hs_6 = np.array(hs_6)

In [8]:
# making arrays of data for input 

wind_16 = []
hs_16 = []

for i in range(len(test_data_16)):
    wind_16.append(test_data_16[i][0].detach().numpy())
    hs_16.append(test_data_16[i][1].detach().numpy())
    
w_16 = np.array(wind_16)
# print(w[1])
hs_16 = np.array(hs_16)

In [9]:
# separating data into batches 
def divide_chunks(l, n):
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
  
batch_size = 8
  
x = list(divide_chunks(w, batch_size))
# print (x[0])

NameError: name 'w' is not defined

In [10]:
# loading model
import sys
# 1 residual block
# model_path = f'/Users/susanbeale/desktop/climate_ml_artifacts/db3c90c54b2e470db9f9bd6c41215eba/artifacts/model'

# 0 residual blocks
# model_path = f'/Users/susanbeale/desktop/climate_ml_artifacts/ec707a55bf66479791d89ef20d66c108/artifacts/model'

# 1 residual block -> coarsened lr input
# model_path = f'/Users/susanbeale/desktop/climate_ml_artifacts/10ea1abccd7d46fda0f8869cb0ccb72f/artifacts/model'

# 16 residual blocks, 3 upsampling layers, coarsened wind input (scale_factor=8)
model_path_16 = f'/Users/susanbeale/desktop/climate_ml_artifacts/70b73c423e4a44f2918feb3b54950a0b/artifacts/model'

# 6 residual blocks, 2 upsampling layers, coarsened wind input (scale_factor=4)
# model_path_6 = f'/Users/susanbeale/desktop/climate_ml_artifacts/6973409397724a43b61a61e03dc7cd70/artifacts/model'


model_16 = mlflow.pyfunc.load_model(model_path_16)

# model_6 = mlflow.pyfunc.load_model(model_path_6)


# model_info = mlflow.models.get_model_info(model_path)


# print(sys.getsizeof(x))
# model.get_params()
torch.cuda.empty_cache()


2023/07/06 13:52:36 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


ModuleNotFoundError: No module named 'ClimatExML.mlflow_tools'

In [ ]:
# pred_6 = model_6.predict(w_6)
pred_16 = model_16.predict(w_16)

In [ ]:
# print(len(x))
print(pred_6.shape)
print(hs_6.shape)

In [ ]:
print(pred_16.shape)
print(hs_16.shape)

In [ ]:
pred_tens_16 = torch.from_numpy(pred_16)
hs_tens_16 = torch.from_numpy(hs_16)
w_tens_16 = torch.from_numpy(w_16)
w_16.shape

In [ ]:
pred_tens_6 = torch.from_numpy(pred_6)
hs_tens_6 = torch.from_numpy(hs_6)
w_tens_6 = torch.from_numpy(w_6)
w_6.shape

In [ ]:
pred_tens.size()[0]

In [ ]:
pred_tens[0].size()

In [ ]:
plt.psd(pred_tens_16, label='fake')
plt.psd(hs_tens_16, label='real')
plt.legend()

In [ ]:
# random selection of truth, generated, and input images
num_row = 5
index = np.random.randint(0,hs_tens_6.size()[0], (num_row))
index = index.astype(int)

row = np.linspace(0,num_row-1, num_row)
row = row.astype(int)

fig, ax = plt.subplots(num_row, 4, figsize=(10, 10))
ax[0,0].set_title('Truth')
ax[0,1].set_title('CNN')
ax[0,2].set_title('Wind Input')


for i,j in zip(index,row):
    ax[j,0].imshow(hs_tens_6[i, 0, ...].detach().cpu(), origin="lower")
    ax[j,1].imshow(pred_tens_6[i, 0, ...].detach().cpu(), origin="lower")
    ax[j,2].imshow(w_tens_6[i, 0, ...].detach().cpu(), origin="lower")
    ax[j,3].imshow(w_tens_6[i, 1, ...].detach().cpu(), origin="lower")

In [ ]:
# random selection of truth, generated, and input images
num_row = 5
index = np.random.randint(0,hs_tens_16.size()[0], (num_row))
index = index.astype(int)

row = np.linspace(0,num_row-1, num_row)
row = row.astype(int)

fig, ax = plt.subplots(num_row, 4, figsize=(10, 10))
ax[0,0].set_title('Truth')
ax[0,1].set_title('CNN')
ax[0,2].set_title('Wind Input')


for i,j in zip(index,row):
    ax[j,0].imshow(hs_tens_16[i, 0, ...].detach().cpu(), origin="lower")
    ax[j,1].imshow(pred_tens_16[i, 0, ...].detach().cpu(), origin="lower")
    ax[j,2].imshow(w_tens_16[i, 0, ...].detach().cpu(), origin="lower")
    ax[j,3].imshow(w_tens_16[i, 1, ...].detach().cpu(), origin="lower")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
i = np.random.randint(0,hs_tens_6.size()[0])
ax[0].imshow(hs_tens_6[i, 0, ...].detach().cpu(), origin="lower")
ax[0].set_title('Truth')
ax[1].imshow(pred_tens_6[i, 0, ...].detach().cpu(), origin="lower")
ax[1].set_title("CNN (6 residual blocks)")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
i = np.random.randint(0,hs_tens_16.size()[0])
ax[0].imshow(hs_tens_16[i, 0, ...].detach().cpu(), origin="lower")
ax[0].set_title('Truth')
ax[1].imshow(pred_tens_16[i, 0, ...].detach().cpu(), origin="lower")
ax[1].set_title("CNN (16 residual blocks)")

In [ ]:
# code from RAPSD

from typing import Generator, Tuple

import numpy as np
import scipy.stats as stats


"""This module contains functions that calculate the
radially averaged power spectral density (RASPD)
"""


def calculate_2dft(image: np.ndarray) -> np.ndarray:
    """Computes the fourier transform and returns the amplitudes"""
    fourier_image = np.fft.fftn(image)
    # fourier_image = torch.fft.fftn(image)
    fourier_amplitudes = np.abs(fourier_image)**2
    # fourier_amplitudes = torch.abs(fourier_image)**2

    return fourier_amplitudes.flatten()


def define_wavenumers(hr_dim: int) -> np.ndarray:
    """Defines the wavenumbers for the RASPD"""
    kfreq = np.fft.fftfreq(hr_dim)*hr_dim
    kfreq2D = np.meshgrid(kfreq, kfreq)
    knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)
    return knrm.flatten()


def get_mean_bins(x, var_idx, knrm) -> Tuple:
    """Calculates the mean bins for the RASPD"""
    kbins = np.arange(0.5, x.shape[-1]//2+1, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    wind_2d = calculate_2dft(x[var_idx, ...].cpu().detach().numpy())
    average_bins, _, _ = stats.binned_statistic(
        knrm, wind_2d, statistic="mean", bins=kbins
        )
    average_bins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)
    return average_bins, kvals


def compute_rapsd(
        hr_field: Generator,
        var_ref: dict = None,
        reduce=True
) -> np.ndarray:
    """Computes the RASPD for a given super-resolution model"""

    if var_ref is None:
        var_ref = {"hs": 0}

    var_rapsd = {}
    [var_rapsd.setdefault(x, []) for x in var_ref]

    for x in hr_field:
        for var_name, var_idx in var_ref.items():
            knrm = define_wavenumers(x.shape[-1])
            average_bins, kvals = get_mean_bins(x, var_idx, knrm)
            var_rapsd[var_name].append(average_bins)

    var_rapsd_avg = {"k": kvals}
    for var_name in var_ref:
        if reduce:
            var_rapsd_avg[var_name] = np.mean(
                np.array(var_rapsd[var_name]),
                axis=0
            )
        else:
            var_rapsd_avg[var_name] = var_rapsd[var_name]

    return var_rapsd_avg


In [ ]:
fake_6 = compute_rapsd(pred_tens_6)
real_6 = compute_rapsd(hs_tens_6)
fake_mean_6 = np.mean(fake_6['hs'])
real_mean_6 = np.mean(real_6['hs'])
norm_fake_6 = fake_6['hs']/fake_mean_6
norm_real_6 = real_6['hs']/real_mean_6


In [ ]:
fake_16 = compute_rapsd(pred_tens_16)
real_16 = compute_rapsd(hs_tens_16)
fake_mean_16 = np.mean(fake_16['hs'])
real_mean_16 = np.mean(real_16['hs'])
norm_fake_16 = fake_16['hs']/fake_mean_16
norm_real_16 = real_16['hs']/real_mean_16

In [ ]:
f,ax = plt.subplots()
# closer to horizontal line is better
# lack of power especially at small spatial scales
# bias???? lack of power 
# ax.plot(fake_6['k'], fake_6['hs']/real_6['hs'], label='fake (6 residual blocks)')
ax.plot(real_16['k'], real_16['hs']/real_16['hs'], label='real')
ax.plot(fake_16['k'], fake_16['hs']/real_16['hs'], label='fake (16 residual blocks)')
# ax.set_xscale('log')
ax.set_yscale('log')
plt.ylabel('RAPSD')
plt.xlabel('wave #')
ax.legend()
ax.grid()

In [ ]:
f,ax = plt.subplots()
# left hand sidfe -> large scale smoothly varying spatial structure/patterns
# to right -> increase spatial frequencies (wave number) -> measuring variability in small scale fluctuations
# ax.plot(fake_6['k'], fake_6['hs'], label='fake (6 residual blocks)')
ax.plot(real_16['k'], real_16['hs'], label='real')
ax.plot(fake_16['k'], fake_16['hs'], label='fake (16 residual blocks)')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()

In [ ]:
import torch.nn.functional as F

mse = F.mse_loss(pred_tens_16, hs_tens_16)
mae = F.l1_loss(pred_tens_16, hs_tens_16)
print('mse: ', mse, 'mae: ', mae)

In [ ]:
hs_tens_16

In [ ]:
# from ClimatExML.wgan_gp import SuperResolutionWGANGP
# from lightning.pytorch.tuner import Tuner
# from lightning.pytorch.loggers import MLFlowLogger

# mlf_logger = MLFlowLogger(experiment_name="ClimatEx WGAN-GP", tracking_uri="sqlite:////Users/susanbeale/desktop/climate_ml_artifacts/climatexdb.sqlite")

# clim_data = ClimatExMLData(data_glob=data, batch_size=1, num_workers=8)

# srmodel = SuperResolutionWGANGP(clim_data)
# trainer = pl.Trainer(precision="32", accelerator="cpu", max_epochs=1, logger=mlf_logger, log_every_n_steps=50)#fast_dev_run=True)
# tuner = Tuner(trainer)

In [ ]:


# trainer.fit(srmodel)

